## The goal of this project is to develop chess program that can play chess and possibly to play it well. 

### Pieces locations in starting position

In [1]:
pieces = {'wR1':bin(2**0), 'wp1':bin(2**8), 'bR1':bin(2**56), 'bp1':bin(2**48),
          'wN1':bin(2**1), 'wp2':bin(2**9), 'bN1':bin(2**57), 'bp2':bin(2**49),
          'wB1':bin(2**2), 'wp3':bin(2**10), 'bB1':bin(2**58), 'bp3':bin(2**50),
          'wQ':bin(2**3), 'wp4':bin(2**11), 'bQ':bin(2**59), 'bp4':bin(2**51),
          'wK':bin(2**4), 'wp5':bin(2**12), 'bK':bin(2**60), 'bp5':bin(2**52),
          'wB2':bin(2**5), 'wp6':bin(2**13), 'bB2':bin(2**61), 'bp6':bin(2**53),
          'wN2':bin(2**6), 'wp7':bin(2**14), 'bN2':bin(2**62), 'bp7':bin(2**54),
          'wR2':bin(2**7), 'wp8':bin(2**15), 'bR2':bin(2**63), 'bp8':bin(2**55),
         }

### Chessboard visualisation

In [2]:
import math
import numpy as np

In [3]:
def set_up_board(pieces_d): # pieces in dictionary
    output = np.chararray((8,8), unicode=True, itemsize=2)
    output[:] = '__'
    for piece in pieces_d.keys():
        num = math.log2(int(pieces_d[piece], 2))+1
        output[(7-int(math.ceil(num/8)-1), int((num-1) % 8))] = piece
    return output

def mark_on_board(fields): #visualisation purposes - find legal moves etc.
    output = np.chararray((8,8), unicode=True, itemsize=2)
    output[:] = '__'
    for field in fields:
        num = math.log2(field)+1
        output[(7-int(math.ceil(num/8)-1), int((num-1) % 8))] = '<>'
    return output

def disp_board(board):
    for i in range(8):
        print('')
        for j in range(8):
            print(board[(i,j)] + ' ', end='')

In [4]:
board = set_up_board(pieces)
disp_board(board)


bR bN bB bQ bK bB bN bR 
bp bp bp bp bp bp bp bp 
__ __ __ __ __ __ __ __ 
__ __ __ __ __ __ __ __ 
__ __ __ __ __ __ __ __ 
__ __ __ __ __ __ __ __ 
wp wp wp wp wp wp wp wp 
wR wN wB wQ wK wB wN wR 

### Pieces moves

In [5]:
def bishop_pattern(placement): 
    
    move_span = []
    num = int(math.log2(placement))
    perm = np.array([1,1,1,1])
    
    #board edges
    lower_b = [0,1,2,3,4,5,6,7]
    upper_b = [56,57,58,59,60,61,62,63]
    left_b = [0,8,16,24,32,40,48,56]
    right_b = [7,15,23,31,39,47,55,63]
    
    #find if ur on the edge of board
    if num in lower_b:
        perm = perm & [1,1,0,0]
        
    if num in upper_b:
        perm = perm & [0,0,1,1]
        
    if num in left_b:
        perm = perm & [0,1,0,1]
        
    if num in right_b:
        perm = perm & [1,0,1,0]
    
    #search in allowed directions
    for i in range(1,8):
        
        if perm[0]:
            up_left = placement << 7*i
            move_span.append(up_left)
            if int(math.log2(up_left)) in upper_b + left_b:
                perm[0] = 0
            
        if perm[1]:
            up_right = placement << 9*i
            move_span.append(up_right)
            if int(math.log2(up_right)) in upper_b + right_b:
                perm[1] = 0
        
        if perm[2]:
            down_left = placement >> 9*i
            move_span.append(down_left)
            if int(math.log2(down_left)) in lower_b + left_b:
                perm[2] = 0
            
        if perm[3]:
            down_right = placement >> 7*i
            move_span.append(down_right)
            if int(math.log2(down_right)) in lower_b + right_b:
                perm[3] = 0
                
    return move_span    

In [6]:
disp_board(mark_on_board(bishop_pattern(2**18)))


__ __ __ __ __ __ __ <> 
__ __ __ __ __ __ <> __ 
__ __ __ __ __ <> __ __ 
<> __ __ __ <> __ __ __ 
__ <> __ <> __ __ __ __ 
__ __ __ __ __ __ __ __ 
__ <> __ <> __ __ __ __ 
<> __ __ __ <> __ __ __ 

In [9]:
bishop_pattern(2**18)

[33554432,
 134217728,
 512,
 2048,
 4294967296,
 68719476736,
 1,
 16,
 35184372088832,
 18014398509481984,
 9223372036854775808]

In [10]:
def rook_pattern(placement):    
    move_span = []
    placenum = int(math.log2(placement))
    start = placenum - placenum % 8
    
    #vertical line
    for i in range(1,8): 
        if placement << 8*i <= 2**63:
            move_span.append(placement << 8*i)
        if placement >> 8*i >= 1:
            move_span.append(placement >> 8*i)
    
    #horizontal line
    for i in range(0,8):
         move_span.append(2**start << i)
        
    return move_span

In [11]:
disp_board(mark_on_board(rook_pattern(2**8)))


<> __ __ __ __ __ __ __ 
<> __ __ __ __ __ __ __ 
<> __ __ __ __ __ __ __ 
<> __ __ __ __ __ __ __ 
<> __ __ __ __ __ __ __ 
<> __ __ __ __ __ __ __ 
<> <> <> <> <> <> <> <> 
<> __ __ __ __ __ __ __ 

In [12]:
def queen_pattern(placement):
    return list(set(rook_pattern(placement) + bishop_pattern(placement)))

In [13]:
disp_board(mark_on_board(queen_pattern(2**43)))


__ <> __ <> __ <> __ __ 
__ __ <> <> <> __ __ __ 
<> <> <> <> <> <> <> <> 
__ __ <> <> <> __ __ __ 
__ <> __ <> __ <> __ __ 
<> __ __ <> __ __ <> __ 
__ __ __ <> __ __ __ <> 
__ __ __ <> __ __ __ __ 

In [14]:
def knight_pattern(placement): 
    
    move_span = []
    num = int(math.log2(placement))
    perm = np.array([1,1,1,1,1,1,1,1])
    
    lower_b = [0,1,2,3,4,5,6,7]
    upper_b = [56,57,58,59,60,61,62,63]
    left_b = [0,8,16,24,32,40,48,56]
    right_b = [7,15,23,31,39,47,55,63]
    
    second = [8,9,10,11,12,13,14,15]
    seventh = [48,49,50,51,52,53,54,55]
    b_file = [1,9,17,25,33,41,49,57]
    g_file = [6,14,22,30,38,46,54,62]
    
    changes=[10,17,15,6,10,17,15,6]
    
    if num in lower_b:
        perm = perm & [0,0,0,0,1,1,1,1]
        
    if num in upper_b:
        perm = perm & [1,1,1,1,0,0,0,0]
        
    if num in left_b:
        perm = perm & [0,0,1,1,1,1,0,0]
        
    if num in right_b:
        perm = perm & [1,1,0,0,0,0,1,1]
        
    if num in lower_b:
        perm = perm & [0,0,0,0,1,1,1,1]
        
    if num in second:
        perm = perm & [1,0,0,1,1,1,1,1]
        
    if num in seventh:
        perm = perm & [1,1,1,1,1,0,0,1]
        
    if num in b_file:
        perm = perm & [0,1,1,1,1,1,1,0]
        
    if num in g_file:
        perm = perm & [1,1,1,0,0,1,1,1]
              
    for i in range(4):
        if perm[i]:
            move_span.append(placement >> changes[i])
    for i in range(4,8):
        if perm[i]:
            move_span.append(placement << changes[i])
            
    return move_span

In [15]:
disp_board(mark_on_board(knight_pattern(2**9)))


__ __ __ __ __ __ __ __ 
__ __ __ __ __ __ __ __ 
__ __ __ __ __ __ __ __ 
__ __ __ __ __ __ __ __ 
<> __ <> __ __ __ __ __ 
__ __ __ <> __ __ __ __ 
__ __ __ __ __ __ __ __ 
__ __ __ <> __ __ __ __ 

In [ ]:
def pawn_pattern(placement):
    
    move_span = []
    lower_b = [0,1,2,3,4,5,6,7]
    upper_b = [56,57,58,59,60,61,62,63]
    second = [8,9,10,11,12,13,14,15]
    seventh = [48,49,50,51,52,53,54,55]
    num = int(math.log2(placement))
    
    #check what color is the pawn
    
    if pawn=='wp':
        if num in second:
            move_span.append(placement << 16)
        if num not in upper_b + lower_b:
            move_span.append(placement << 8)
            
    if pawn=='bp':
        if num in seventh:
            move_span.append(placement << 16)
        if num not in upper_b + lower_b:
            move_span.append(placement << 8)
    

In [16]:
pieces(2**10)

TypeError: 'chararray' object is not callable